In [1]:
import pandas as pd

### Data cleaning

In [2]:
df = pd.read_csv('data/MaricopaCountyNewHousePermits.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4573 entries, 0 to 4572
Data columns (total 19 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   DESCRIPTION                        4573 non-null   object 
 1   CATEGORY                           4573 non-null   object 
 2   TRACKING NUMBER                    4573 non-null   object 
 3   ISSUE DATE                         4573 non-null   object 
 4   THIS PERMIT IS FOR                 4573 non-null   object 
 5   VALUATION                          4573 non-null   float64
 6   BUILDING FOOTPRINT SQUARE FOOTAGE  4518 non-null   float64
 7   PARCEL NO                          4568 non-null   object 
 8   NO UNITS                           4573 non-null   int64  
 9   JOB ADDRESS                        4568 non-null   object 
 10  SUBDIVISION                        2161 non-null   object 
 11  LOT                                2481 non-null   objec

In [3]:
df.loc[:,'JOB ZIP'] = df['JOB ZIP'].astype('str')
df.loc[:,'CONTACTOR PHONE NUMBER'] = df['CONTACTOR PHONE NUMBER'].astype('str')

### Get supplemental information for owner entities

In [4]:
ownerCounts = pd.DataFrame(df['OWNER NAME'].value_counts())
# Get owner names for owners that own at least five homes
ownerCos = ownerCounts.loc[ownerCounts['count'] >= 5].index.to_list()

In [5]:
ownerCos = df.loc[df['OWNER NAME'].isin(ownerCos),['OWNER NAME', 'TRACKING NUMBER']].groupby('OWNER NAME').first()
ownerCos.head(1)

,TRACKING NUMBER
OWNER NAME,
110 HOLDINGS LLC,B202111384


In [6]:
from bs4 import BeautifulSoup
from requests import get 

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
permitNos = []
names = []
cos = []
address1 = []
address2 = []
regions = []
countries = []
phones = []
emails = []

def search(permit):
    driver.get('https://accela.maricopa.gov/CitizenAccessMCOSS/Cap/CapHome.aspx?module=PnD&TabName=PnD')
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, 'Search')))
    driver.find_element(By.XPATH, '//*[@id="ctl00_PlaceHolderMain_generalSearchForm_txtGSPermitNumber"]').clear()
    driver.find_element(By.XPATH, '//*[@id="ctl00_PlaceHolderMain_generalSearchForm_txtGSPermitNumber"]').send_keys(permit)
    driver.find_element(By.LINK_TEXT, 'Search').click()

def getPage():
    # Get HTML
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'ctl00_PlaceHolderMain_lblPermitNumber')))
    page = driver.page_source
    soup = BeautifulSoup(page)
    return soup

def getOwnerInfo(soup, permit):
    permitNos.append(permit)
    applicant = soup.find(class_='ACA_TabRow ACA_FLeft')
    # spans = applicant.find_all('span')

    try:
        names.append(applicant.find(class_='contactinfo_firstname').text.strip() + ' ' + applicant.find(class_='contactinfo_lastname').text.strip())
    except:
        names.append('')
    try:
        cos.append(applicant.find(class_='contactinfo_businessname').text.strip())
    except:
        cos.append('')
    try:
        address1.append(applicant.find(class_='contactinfo_addressline1').text.strip())
    except:
        address1.append('')
    try:
        address2.append(applicant.find(class_='contactinfo_addressline2').text.strip())
    except:
        address2.append('')
    try:
        rs = applicant.find_all(class_='contactinfo_region')
        regions.append(''.join([x.text for x in rs]))
    except:
        regions.apend('')
    try:
        countries.append(applicant.find(class_='contactinfo_country').text.strip())
    except:
        countries.append('')
    try:
        phones.append(applicant.find(class_='ACA_PhoneNumberLTR').text.strip())
    except:
        phones.append('')
    try:
        emails.append(applicant.find(class_='contactinfo_email').text.strip())
    except:
        emails.append('')

def getInfo(permit, i):
    search(permit)
    soup = getPage()
    getOwnerInfo(soup, permit)
    print(f'{i}: {permit}')

driver = webdriver.Chrome()
driver.implicitly_wait(10)
i = 1
for permit in ownerCos['TRACKING NUMBER']:
    getInfo(permit, i)
    i += 1

driver.close()

1: B202111384
2: B202213560
3: B202111861
4: B202215742
5: B202302247
6: B202302010
7: B202216205
8: B202203315
9: B202117520
10: B202306779
11: B202200537
12: B202300813
13: B202304827
14: B202200711
15: B202211220
16: B202201195
17: B202202130
18: B202212304
19: B202213467
20: B202204717
21: B202215161
22: B202116768
23: B202204164
24: B202212067
25: B202109295
26: B202303024
27: B202200618
28: B202116347
29: B202304753
30: B202117477
31: B202105430
32: B202205424
33: B202201263
34: B202306960
35: B202215669
36: B202115285
37: B202305982
38: B202206422
39: B202205095
40: B202305751
41: B202200123
42: B202306559
43: B202215024
44: B202207083
45: B202303977
46: B202303880
47: B202114957
48: B202201645
49: B202201024
50: B202202751
51: B202304087
52: B202303364
53: B202306464
54: B202305261
55: B202206565
56: B202306698
57: B202204281
58: B202216864
59: B202209359
60: B202214994
61: B202306832
62: B202215862
63: B202216284
64: B202203745
65: B202216250
66: B202306065
67: B202208107


In [8]:
ownerInfo = pd.DataFrame(
dict(zip(['Permit', 'Owner Name', 'Business Name', 'Business Address1', 'Business Address2', 'Business Region',
 'Business Country', 'Contact Name', 'Phone', 'Email'],
[permitNos, ownerCos.index.to_list(), cos, address1, address2, regions, countries, names, phones, emails])))

In [9]:
ownerInfo.head()

,Permit,Owner Name,Business Name,Business Address1,Business Address2,Business Region,Business Country,Contact Name,Phone,Email
0,B202111384,110 HOLDINGS LLC,ARQM LLC,2201 S 112th Ave,,"Avondale, AZ, 85323",United States,Rosalio Mondragon,6232533751,Arqmarchitect@outlook.com
1,B202213560,2PHDS LLC,2PHDS LLC,8341 E Gelding Dr,,"Scottsdale, AZ, 85260",United States,Andrei Polukhtin,3157293279,morningvistahomes@gmail.com
2,B202111861,A AND B HOMES INC,A and B Homes Inc,8826 W Alice Ave,,"Peoria, AZ, 85345",United States,Phillip Woolbright,9287934346,phillipwoolbright@gmail.com
3,B202215742,AG EHC II LEN MULTI STATE 2 LLC,Lennar,1665 W Alameda Dr,,"Tempe, AZ, 85282",United States,Cory Calhoun,4807601249,cory.calhoun@lennar.com
4,B202302247,AMH DEVELOPMENT LLC,"AMH Development West GC, LLC",4050 E Cotton Center Blv,Suite 70,"Phoenix, AZ, 85040",United States,Tanya Daniels,4803295720,tdaniels@ah4r.com


In [10]:
ownerInfo['Business Name'].unique()

array(['ARQM LLC', '2PHDS LLC', 'A and B Homes Inc', 'Lennar',
       'AMH Development West GC, LLC', 'Ashton Woods Homes',
       'Diamante Homes Inc', 'Beazer Homes', 'Morgan Taylor Homes',
       'TLD Builders, LLC', 'K. Hovnanian Homes, LLC',
       'Bungalows on Cotton Lane LLC', '', 'COURTLAND COMMUNITIES LLC',
       'Ingram Design Build', 'Elliott Homes, Inc.', 'BLUEPRINT & DESIGN',
       'Landsea Homes', 'Gehan Homes of Arizona, LLC',
       'HBT Construction of AZ, Inc.', 'David Weekley Homes',
       'JW Builders INC', 'PHK Devlopment LLC', 'KB Home',
       'Stellar Contracting', 'Craft Development LLC',
       'Monster Pool Company LLC', 'Meritage Homes Corporation',
       'Newmark Homes', 'Nexstar Homes', 'Rd Architectural Consultants',
       'Longboard Home Services, LLC', 'Copper Sky Homes',
       'Richmond American Homes', 'Scott Communities',
       'William Ryan Homes Arizona, Inc', 'Taylor Morrison',
       'KM Development Corp', 'NextGen Builders LLC', 'Shea Ho

In [11]:
ownerInfo['Business City'] = [x.split(', ')[0].capitalize() for x in ownerInfo['Business Region']]
ownerInfo['Business State'] = [x.split(', ')[1].upper() for x in ownerInfo['Business Region']]
ownerInfo['Business Zipcode'] = [x.split(', ')[2] for x in ownerInfo['Business Region']]

In [12]:
ownerInfo.loc[:,'Phone'] = [f'{x[:3]}-{x[3:6]}-{x[6:]}' for x in ownerInfo.Phone]
ownerInfo.rename(columns={'Owner Name':'OWNER NAME'}, inplace=True)

### Merge full data with owner information

In [13]:
merged = df.merge(ownerInfo[['OWNER NAME', 'Business Name', 'Business Address1', 'Business Address2', 'Business City', 'Business State', 'Business Zipcode', 'Contact Name', 'Phone', 'Email']], on='OWNER NAME')
merged.head()

,DESCRIPTION,CATEGORY,TRACKING NUMBER,ISSUE DATE,THIS PERMIT IS FOR,VALUATION,BUILDING FOOTPRINT SQUARE FOOTAGE,PARCEL NO,NO UNITS,JOB ADDRESS,...,CONTRACTOR EMAIL,Business Name,Business Address1,Business Address2,Business City,Business State,Business Zipcode,Contact Name,Phone,Email
0,Residential,Production,B202200618,2023-07-19,LOT 82 SP20210096 SFR PLAN 2012 ELEV A MIRROR ...,187691.00,2591.0,502-40-584,1,"19463 W ANNIKA Dr\nLITCHFIELD PARK, AZ 85340",...,mmoskal@gehanhomes.com,"Gehan Homes of Arizona, LLC",1501 W Fountainhead Pkwy/. Ste 150,,Tempe,AZ,85282,Candy Adamow,480-570-9481,cadamow@gehanhomes.com
1,Residential,Production,B202306400,2023-07-10,SP20220178 // PRODUCTION NEW SFR w/ 258LF CMU ...,160019.08,2209.0,502-40-551,1,"19431 W SOLANO Dr\nLITCHFIELD PARK, AZ 85340",...,CADAMOW@GEHANHOMES.COM,"Gehan Homes of Arizona, LLC",1501 W Fountainhead Pkwy/. Ste 150,,Tempe,AZ,85282,Candy Adamow,480-570-9481,cadamow@gehanhomes.com
2,Residential,Production,B202306406,2023-07-11,SP20220180 // PRODUCTION SFR W/FENCE // PLAN 1...,185662.69,2563.0,502-40-550,1,"19437 W SOLANO Dr\nLITCHFIELD PARK, AZ 85340",...,CADAMOW@GEHANHOMES.COM,"Gehan Homes of Arizona, LLC",1501 W Fountainhead Pkwy/. Ste 150,,Tempe,AZ,85282,Candy Adamow,480-570-9481,cadamow@gehanhomes.com
3,Residential,Production,B202306494,2023-07-11,SP20220179 SINGLE FAMILY RESIDENCE; PLAN 1680;...,168929.15,2332.0,502-40-614,1,"19309 W SOLANO Dr\nLITCHFIELD PARK, AZ 85340",...,CADAMOW@GEHANHOMES.COM,"Gehan Homes of Arizona, LLC",1501 W Fountainhead Pkwy/. Ste 150,,Tempe,AZ,85282,Candy Adamow,480-570-9481,cadamow@gehanhomes.com
4,Residential,Production,B202306567,2023-07-11,SP20220180 SINGLE FAMILY RESIDENCE; PLAN 1921;...,185662.69,2563.0,502-40-613,1,"19303 W SOLANO Dr\nLITCHFIELD PARK, AZ 85340",...,CADAMOW@GEHANHOMES.COM,"Gehan Homes of Arizona, LLC",1501 W Fountainhead Pkwy/. Ste 150,,Tempe,AZ,85282,Candy Adamow,480-570-9481,cadamow@gehanhomes.com


### Save merged and grouped data to Excel

In [19]:
# Address blank business names
merged.loc[(merged['Business Name']=='')&(merged['Email']!='ghan5150@hotmail.com'), 'Business Name'] = merged.loc[(merged['Business Name']=='')&(merged['Email']!='ghan5150@hotmail.com'), 'OWNER NAME']
merged.loc[(merged['Business Name']=='')&(merged['Email']=='ghan5150@hotmail.com'), 'Business Name'] = 'ghan5150'

In [20]:
merged.loc[merged['Business Name']!='', ['Business Name', 'VALUATION', 'NO UNITS', 'Contact Name', 'Phone', 'Email']]\
    .groupby('Business Name')\
    .aggregate({'VALUATION':'sum', 'NO UNITS':'count', 'Contact Name':'first', 'Phone':'first', 'Email':'first'})\
    .sort_values('NO UNITS', ascending=False)\
    .rename(columns={'VALUATION':'Total Permitted Project Costs', 'NO UNITS':'Total Permitted Units'})\
    .to_excel('data/MaricopaHomes.xlsx', sheet_name='MaricopaHomeCos')

In [22]:
ownerInfo.to_csv('data/ownerInfo.csv')

In [23]:
merged.to_csv('data/full.csv')